In [1]:
from __future__ import absolute_import, division, print_function

# 使用 CPU，不使用GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 导入 tensorflow >=1.10 和 使用 eager execution
# eager execution: 提供了 命令高级操作接口，通过自动微分编写自定义层、前向传播和训练循环
import tensorflow as tf
tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.12.0


In [2]:
# 下载预训练数据集
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://download.tensorflow.org/data/spa-eng.zip', 
    extract=True)
path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [3]:
# 转换unicode 文件为 ascii
# unicodedata.normalize() 第一个参数指定字符串标准化的方式。 
# NFC表示字符应该是整体组成(比如可能的话就使用单一编码)，而NFD表示字符应该分解为多个组合字符表示
# unicodedata.category(c): 分配给字符c的常规类别为字符串。
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


# 预处理句子,删除空白字符
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [4]:
def create_dataset(path, num_examples):
    lines = open(path, encoding = 'utf-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return word_pairs

In [5]:
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index(), 

    # 创建语言对应索引数值，方便检索
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))
        self.vocab = sorted(self.vocab)
        
        self.word2idx['pad'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1
        
        for word, index in self.word2idx.items():
            self.idx2word[index] = word
        

In [6]:
# 提取最大的张量值
def max_length(tensor):
    return max(len(t) for t in tensor)

In [7]:
# 加载数据集
def load_dataset(path, num_examples):
    pairs = create_dataset(path, num_examples)
    print(pairs)
    # 建立索引
    inp_lang = LanguageIndex(sp for en, sp in pairs)
    targ_lang = LanguageIndex(en for en, sp in pairs)
    
    # 否量化输入和目标语言
    # 西班牙语
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    # 英语
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    
    # 计算最大的输入和目标值
    max_length_inp, max_length_targ = max_length(input_tensor), max_length(target_tensor)
    
    # 填充输入和目标值张量最大值
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                                 maxlen=max_length_targ,
                                                                 padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ

In [8]:
# 限制数量，数量太多对> 100,000个句子的完整数据集进行培训需要很长时间。
# 为了更快地训练，我们可以将数据集的大小限制为30,000个句子（当然，翻译质量会随着数据的减少而降低）：
num_examples = 500
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

[['<start> go . <end>', '<start> ve . <end>'], ['<start> go . <end>', '<start> vete . <end>'], ['<start> go . <end>', '<start> vaya . <end>'], ['<start> go . <end>', '<start> vayase . <end>'], ['<start> hi . <end>', '<start> hola . <end>'], ['<start> run ! <end>', '<start> corre ! <end>'], ['<start> run . <end>', '<start> corred . <end>'], ['<start> who ? <end>', '<start> ¿ quien ? <end>'], ['<start> fire ! <end>', '<start> fuego ! <end>'], ['<start> fire ! <end>', '<start> incendio ! <end>'], ['<start> fire ! <end>', '<start> disparad ! <end>'], ['<start> help ! <end>', '<start> ayuda ! <end>'], ['<start> help ! <end>', '<start> socorro ! auxilio ! <end>'], ['<start> help ! <end>', '<start> auxilio ! <end>'], ['<start> jump ! <end>', '<start> salta ! <end>'], ['<start> jump . <end>', '<start> salte . <end>'], ['<start> stop ! <end>', '<start> parad ! <end>'], ['<start> stop ! <end>', '<start> para ! <end>'], ['<start> stop ! <end>', '<start> pare ! <end>'], ['<start> wait ! <end>', '<

In [9]:
#print(input_tensor)
#print(inp_lang)

In [10]:
# 创建数据集分为训练集和验证集比例 8:2
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size = 0.2)

# 显示长度
len(input_tensor_train), len(input_tensor_val), len(target_tensor_train), len(target_tensor_val)

(400, 100, 400, 100)

In [11]:
# tf.data 数据集
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE / BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

# 把数据集随机分片处理
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((64, 8), (64, 6)), types: (tf.int32, tf.int32)>


In [12]:
def gru(units):
    return tf.keras.layers.GRU(units,
                              return_sequences=True,
                              return_state=True,
                              recurrent_activation='sigmoid',
                              recurrent_initializer='glorot_uniform')

In [21]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_sz = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [33]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_sz = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hiddend, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        # hiddend shape ==(batch_size, hidden_size)
        # 将尺寸1插入张量的形状,比如 shape [2] 插入后为 [2, 1]
        print(hidden)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        print(hidden_with_time_axis)
        
        # score shape == (batch_size, max_length, 1)
        # 使用激活函数 tanh(FC(EO) + FC(H)) 
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # 文本词向量
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        #将连接的向量传递给GRU
        ouput, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    # 初始化
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))     

In [34]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)